# Recommendation System: Retrieval Stage

Retrieval models are often composed of two sub-models:

- A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
- A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

In [1]:
# Import packages
import os
import numpy as np
import tensorflow as tf
from pprint import pprint
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

tf.__version__

'2.7.0'

## Prepare data

In [2]:
os.listdir("/database/tensorflow-datasets/")

['movielens', 'datasets', 'tiny_shakespeare', 'imdb_reviews', 'downloads']

In [3]:
# Load data
ratings = tfds.load("movielens/100k-ratings", split="train", data_dir="/database/tensorflow-datasets/")
movies = tfds.load("movielens/100k-movies", split="train", data_dir="/database/tensorflow-datasets/")

2021-12-16 21:55:18.047556: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 21:55:18.052587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 21:55:18.053178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 21:55:18.054093: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
for x in ratings.take(1).as_numpy_iterator():
	pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


2021-12-16 21:55:18.794950: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


There are a couple of key features here:

- Movie title is useful as a movie identifier
- Movie genre
- User id is useful as a user identifier
- User occupation label
- User gender
- Timestamps will allow us to model the effect of time

In [5]:
for x in movies.take(1).as_numpy_iterator():
	pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


2021-12-16 21:55:18.901218: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [13]:
# Select a subset of features
ratings = ratings.map(lambda x: {
	"movie_title": x["movie_title"],
	"user_id": x["user_id"],
	"user_occupation_text": x["user_occupation_text"],
	"timestamp": x["timestamp"]
})
movies = movies.map(lambda x: x["movie_title"])

In [14]:
# Create train and test set (ideally based on time) using random split
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [15]:
for x in train.take(1).as_numpy_iterator():
	pprint(x)

{'movie_title': b'Postman, The (1997)',
 'timestamp': 885409515,
 'user_id': b'681',
 'user_occupation_text': b'marketing'}


In [16]:
for x in test.take(1).as_numpy_iterator():
	pprint(x)

{'movie_title': b'M*A*S*H (1970)',
 'timestamp': 874948475,
 'user_id': b'346',
 'user_occupation_text': b'other'}


In [18]:
# Get unique movies and user_id present in the data
# movie_titles = movies.batch(1000).map(lambda x: x["movie_title"])
# user_occupation_text = ratings.batch(1000).map(lambda x: x["user_occupation_text"])
# user_ids = ratings.batch(1_000_00).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_occupation_text = np.unique(np.hstack(list(ratings.batch(1_000).map(lambda x: x["user_occupation_text"]))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(lambda x: x["user_id"]))))

print(unique_user_ids[:5])
print(unique_movie_titles[:5])
print(unique_user_occupation_text[:5])

[b'1' b'10' b'100' b'101' b'102']
[b"'Til There Was You (1997)" b'1-900 (1994)' b'101 Dalmatians (1996)'
 b'12 Angry Men (1957)' b'187 (1997)']
[b'administrator' b'artist' b'doctor' b'educator' b'engineer']


## Implement model

Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

In [21]:
# Create timestamp buckets as timestamps in general are too large to be used with any deep learning model
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))
max_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(tf.cast(0, tf.int64), tf.maximum).numpy().max()
min_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(np.int64(1e9), tf.minimum).numpy().min()

timestamp_buckets = np.linspace(min_timestamp, max_timestamp, num=1000)

print(f"Buckets: {timestamp_buckets[:3]}")

Buckets: [8.74724710e+08 8.74743291e+08 8.74761871e+08]


In [22]:
class UserModel(tf.keras.Model):
	def __init__(self) -> None:
		super().__init__()
		self.user_embedding = tf.keras.Sequential(
			[
				tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
				tf.keras.layers.Embedding(len(unique_user_ids)+1, 64)
			]
		)
		self.occupation_text_embedding = tf.keras.Sequential(
			[
				tf.keras.layers.StringLookup(vocabulary=unique_user_occupation_text, mask_token=None),
				tf.keras.layers.Embedding(len(unique_user_occupation_text)+1, 31)
			]
		)
		self.timestamp_embedding = tf.keras.Sequential(
			[
				tf.keras.layers.Discretization(timestamp_buckets.tolist()),
				tf.keras.layers.Embedding(len(timestamp_buckets) + 2, 32)
			]
		)
		self.timestamp_normalization = tf.keras.layers.Normalization(axis=None)
		self.timestamp_normalization.adapt(timestamps)
	
	def call(self, inputs):
		return tf.concat([
			self.user_embedding(inputs["user_id"]),
			self.occupation_text_embedding(inputs["user_occupation_text"]),
			self.timestamp_embedding(inputs["timestamp"]),
			tf.reshape(self.timestamp_normalization(inputs["timestamp"]), (-1, 1))
			], axis=1)

In [23]:
# Lets try it out
user_model = UserModel()

In [30]:
class MovieModel(tf.keras.Model):
	def __init__(self) -> None:
		super().__init__()
		self.title_embedding = tf.keras.Sequential(
			[
				tf.keras.layers.StringLookup(vocabulary=unique_movie_titles, mask_token=None),
				tf.keras.layers.Embedding(len(unique_movie_titles)+1, 64)
			]
		)
		self.title_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000)
		self.title_text_embedding = tf.keras.Sequential(
			[
				self.title_vectorizer,
				tf.keras.layers.Embedding(10000, 64, mask_zero=True),
				tf.keras.layers.GlobalAveragePooling1D()
			]
		)
		self.title_vectorizer.adapt(movies)
	
	def call(self, movie_title):
		return tf.concat([
			self.title_embedding(movie_title),
			self.title_text_embedding(movie_title)
			], axis=1)

In [31]:
# Lets try it out
movie_model = MovieModel()

In our training data we have positive (user, movie) pairs. 

To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

In [33]:
# Set metric
metrics = tfrs.metrics.FactorizedTopK(candidates=movies.batch(128).map(movie_model))

In [34]:
# Set objective
retrieval_task = tfrs.tasks.Retrieval(metrics=metrics)

The task itself is a keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

In [35]:
# Combine the candidate and user model to build the complete retrieval model
class MovielensModel(tfrs.Model):
	def __init__(self, user_model, movie_model, retrieval_task):
		super().__init__()
		self.movie_model = movie_model
		self.user_model = user_model
		self.task = retrieval_task
	
	def compute_loss(self, features, training=False):
		embedding_1 = self.user_model({
			"user_id": features["user_id"],
			"user_occupation_text": features["user_occupation_text"],
			"timestamp": features["timestamp"]
		})
		embedding_2 = self.movie_model(features["movie_title"])
		return self.task(embedding_1, embedding_2)

The tfrs.Model base class is a simply convenience class: it allows us to compute both training and test losses using the same method.

## Learn and evaluate model

In [36]:
# Get combined model
main_model = MovielensModel(user_model, movie_model, retrieval_task)

# Compile model
main_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))

Now shuffle, batch and cache training and evaluation data.

In [37]:
cached_train = train.shuffle(100_000).batch(4096).cache()
cached_test = test.batch(4096).cache()

In [38]:
# Train model
main_model.fit(
	cached_train, epochs=20, validation_data=cached_test
)

Epoch 1/20
20/20 [==============================] - 7s 258ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7500e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0043 - factorized_top_k/top_10_categorical_accuracy: 0.0091 - factorized_top_k/top_50_categorical_accuracy: 0.0448 - factorized_top_k/top_100_categorical_accuracy: 0.0840 - loss: 32402.9842 - regularization_loss: 0.0000e+00 - total_loss: 32402.9842 - val_factorized_top_k/top_1_categorical_accuracy: 0.0014 - val_factorized_top_k/top_5_categorical_accuracy: 0.0080 - val_factorized_top_k/top_10_categorical_accuracy: 0.0153 - val_factorized_top_k/top_50_categorical_accuracy: 0.0625 - val_factorized_top_k/top_100_categorical_accuracy: 0.1102 - val_loss: 29596.5801 - val_regularization_loss: 0.0000e+00 - val_total_loss: 29596.5801
Epoch 2/20
20/20 [==============================] - 4s 195ms/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top

In [39]:
# Evaluate model
main_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 160ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0120 - factorized_top_k/top_10_categorical_accuracy: 0.0268 - factorized_top_k/top_50_categorical_accuracy: 0.1418 - factorized_top_k/top_100_categorical_accuracy: 0.2582 - loss: 31152.1572 - regularization_loss: 0.0000e+00 - total_loss: 31152.1572


{'factorized_top_k/top_1_categorical_accuracy': 0.0010999999940395355,
 'factorized_top_k/top_5_categorical_accuracy': 0.011950000189244747,
 'factorized_top_k/top_10_categorical_accuracy': 0.026750000193715096,
 'factorized_top_k/top_50_categorical_accuracy': 0.14184999465942383,
 'factorized_top_k/top_100_categorical_accuracy': 0.25824999809265137,
 'loss': 28324.37109375,
 'regularization_loss': 0,
 'total_loss': 28324.37109375}

In this model, we created a user-movie model. However, for some applications (for example, product detail pages) it's common to perform item-to-item (for example, movie-to-movie or product-to-product) recommendations.